In [1]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [2]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.analyses.sweep_options.sweeping import Sweeping
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [3]:
# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
gui = MetalGUI(design)

# Set hfss as renderer for design
hfss = design.renderers.hfss

# Allow overwriting
design.overwrite_enabled = True


In [5]:
# Add hanger of capacitievely coupled transmission lines
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='0mm',
                                                 pos_y='1.3mm',
                                                 coupling_length='100um'))
gui.rebuild()
gui.autoscale()

In [6]:
# Create shorted pin and open pin for resonator
ops = dict(fillet='90um')
otg3 = OpenToGround(design, 'open_to_ground3', options=dict(pos_x='0mm', pos_y='-1.2mm',
orientation='270'))

route_options= Dict(total_length='8mm',
                   hfss_wire_bonds=False,
                   pin_inputs=Dict(start_pin=Dict(component='TQ1',
                                                 pin='second_end'),
                                  end_pin=Dict(component=otg3.name, pin='open')),
                   lead=Dict(start_straight='0.1mm'),
                   **ops)

# Create meandered resonator between shorted and opened pin
rt_meander = RouteMeander(design, 'readout', options=route_options)

gui.rebuild()
gui.autoscale()


In [7]:
#Setup the launchpad location and orientation.
#launch_options1 = dict(pos_x='-500um', pos_y='0.5mm', orientation='0')

#lp = LaunchpadWirebondCoupled(design, 'P4_Q', options = launch_options1)

otg = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='2mm', pos_y='1.3mm',
orientation='0'))



#Setup the OpenToGround location and orientation.
#launch_options2 = dict(pos_x='0.75mm',  pos_y='0.5mm', orientation='180')

#lp2 = LaunchpadWirebondCoupled(design, 'P5_Q', options = launch_options2)


otg2 = OpenToGround(design, 'open_to_ground', options=dict(pos_x='-2mm', pos_y='1.3mm',
orientation='180'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options_left = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_start'),
            end_pin=Dict(component=otg2.name, pin='open')), )

meander_options_right = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_end'),
            end_pin=Dict(component=otg.name, pin='open')), )


meander_left = RouteStraight(design, 'cpw_left',options=meander_options_left)
meander_right = RouteStraight(design, 'cpw_right',options=meander_options_right)

gui.rebuild()
gui.autoscale()

In [ ]:
# Screenshot the canvas only as a .png formatted file.
#gui.figure.savefig('shot.png')

#from IPython.display import Image, display
#_disp_ops = dict(width=500)
#display(Image('shot.png', **_disp_ops))

In [8]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [ ]:
hfss.connect_ansys()

In [9]:
sweep = Sweeping(design)

In [10]:
hfss.options['x_buffer_width_mm'] = 0.5
hfss.options['y_buffer_width_mm'] = 0.5

In [11]:
selection = [] #render everything in geometry table
open_pins = []
port_list = [('cpw_right', 'end', 50), ('cpw_left', 'end', 50)]
jj_to_port = []
ignored_jjs = []
box_plus_buffer = True

dm_render_args = Dict(selection=selection,
                      open_pins=open_pins,
                      port_list=port_list,
                      jj_to_port=jj_to_port,
                      ignored_jjs=ignored_jjs,
                      box_plus_buffer=box_plus_buffer)


In [ ]:
hfss.clean_active_design()

In [ ]:
hfss.render_design([],
                   [],
                  [('cpw_right', 'end', 50), ('cpw_left', 'end', 50)],
                  [],
                  [],
                  box_plus_buffer=False)

In [ ]:
design._chips['main']['size']['size_x'] = '3mm'
design._chips['main']['size']['size_y'] = '3mm'

In [12]:
# Pass a dict with arguments to be used add Setup to project.
# If you don't pass all the arguments, the default is determined by
# QHFSSRenderer's default_options.
# The name of setup will be "Sweep_dm_setup".
# If a setup named "Sweep_dm_setup" exists in the project, it will be deleted,
# and a new setup will be added with the arguments from setup_args.
hfss_dm_setup_args = Dict(freq_ghz=None,
                          max_delta_s=None,
                          max_passes=None,
                          min_passes=None,
                          min_converged=None,
                          pct_refinement=None,
                          basis_order=None)
# Example of updating the keys that we are interested in.
hfss_dm_setup_args = Dict(
    freq_ghz=6.0,
    max_delta_s=0.05,
    max_passes=12,
    min_passes=2,
)

In [13]:
dm_add_sweep_args = Dict(name="Sweep_options__dm_sweep",
                         start_ghz=3.55,
                         stop_ghz=3.75,
                         count=16001,
                         type="Interpolating")

In [14]:
all_sweeps, return_code = sweep.sweep_one_option_get_drivenmodal_solution_data(
    'readout',
    'total_length', ['9mm', '8mm', '7mm'],
    dm_render_args,
    setup_args=hfss_dm_setup_args,
    dm_add_sweep_args=dm_add_sweep_args,
    leave_last_design=True)

INFO 06:39PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:39PM [load_ansys_project]: 	Opened Ansys App
INFO 06:39PM [load_ansys_project]: 	Opened Ansys Desktop v2019.1.0
INFO 06:39PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/2254729M/Documents/Ansoft/
	Project:   Project6
INFO 06:39PM [connect_design]: No active design found (or error getting active design).
INFO 06:39PM [connect]: 	 Connected to project "Project6". No design detected
06:39PM 27s WARNING [activate_ansys_design]: The design_name=Sweep_DrivenModal was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 06:39PM [connect_design]: 	Opened active design
	Design:    Sweep_DrivenModal [Solution type: DrivenModal]
WARNING 06:39PM [connect_setup]: 	No design setup detected.
WARNING 06:39PM [connect_setup]: 	Creating drivenmodal default setup.
INFO 06:39PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 

In [ ]:
# Each key corresponds to list passed to ['9mm', '8mm', '7mm']
if return_code == 0:
    print(all_sweeps.keys())
else:
    print('Check warning messages to see why all_sweeps is non-zero.')

In [ ]:
all_sweeps['8mm']['option_name']

In [ ]:
# For each key, know if the solution data converged.
if return_code == 0:
    for item in all_sweeps.keys():
        option_name = all_sweeps[item]['option_name']
        did_converge = all_sweeps[item]['convergence']
        print(f'For {option_name}={item}, convergence is {did_converge}.')
else:
    print('Check warning messages to see why all_sweeps is non-zero.')

In [ ]:
# For example, just one group of solution data. Look at convergence data.
if return_code == 0:
    print(all_sweeps['8mm']['convergence_t'])
else:
    print('Check warning messages to see why all_sweeps is non-zero.')

In [ ]:
# Each values use for the given option, has three matrixes:  
# scatter, impedance, and admittance.
if return_code == 0:
    print(all_sweeps['8mm'].keys())
else:
    print('Check warning messages to see why all_sweeps is non-zero.')

In [ ]:
# For example, just one group of solution data. 
if return_code == 0:
    print(all_sweeps['8mm']['z_matrix'])
else:
    print('Check warning messages to see why all_sweeps is non-zero.')

In [ ]:
if return_code == 0:
    df_s = all_sweeps['8mm']['s_matrix']
else:
    print('Check warning messages to see why all_sweeps is non-zero.')

In [ ]:
df_s['20_log_of_mag_S11']= 20 * np.log10(np.absolute(df_s['S11']))
df_s['20_log_of_mag_S21']= 20 * np.log10(np.absolute(df_s['S21']))
#df_s['20_log_of_mag_S31']= 20 * np.log10(np.absolute(df_s['S31']))

# Reference to current axis. 
magnitude = plt.figure('Magnitude S11, S21, and S31')
plt.clf()
axis = plt.gca() # Get current axis.
df_s = all_sweeps['8mm']['s_matrix']
df_s.plot(kind = 'line', y='20_log_of_mag_S11', color = 'green', ax = axis)
df_s.plot(kind = 'line', y='20_log_of_mag_S21', color = 'blue', ax = axis)
#df_s.plot(kind = 'line', y='20_log_of_mag_S31', color = 'red', ax = axis)
plt.title(f'S-Parameter Magnitude')
plt.xlabel(f'frequency [GHZ]')
plt.ylabel(f'|S11|,|S21| [dB]')
magnitude.show()


In [ ]:
# Data is shown as degrees.  
# However, if you want radians, change value of deg to false, deg=False.
df_s['degrees_S11'] = np.angle(df_s['S11'], deg=True)
df_s['degrees_S21'] = np.angle(df_s['S21'], deg=True)
#df_s['degrees_S31'] = np.angle(df_s['S31'], deg=True)

# Reference to current axis. 
phase = plt.figure('Phase of S11 and S21')
plt.clf()
axis = plt.gca() # Get current axis.
df_s = all_sweeps['8mm']['s_matrix']
df_s.plot(kind = 'line', y='degrees_S11', color = 'green', ax = axis)
df_s.plot(kind = 'line', y='degrees_S21', color = 'blue', ax = axis)
#df_s.plot(kind = 'line', y='degrees_S31', color = 'red', ax = axis)
plt.title(f'S-Parameter Phase')
plt.xlabel(f'frequency [GHZ]')
plt.ylabel(f'<S11, <S21, <S31 [degrees]')
phase.show()

In [ ]:
# In HFSS, after the last sweep, should the design be cleared?  The argument "leave_last_design" is true by default. 
# If you want to delete after last sweep, change argument to false.

leave_last_design = False

all_sweeps, return_code = sweep.sweep_one_option_get_drivenmodal_solution_data(
    meanderQ1.name,
    'total_length', ['9mm', '8mm', '7mm'],
    dm_render_args,
    setup_args=hfss_dm_setup_args,
    dm_add_sweep_args=dm_add_sweep_args,
    leave_last_design=leave_last_design)

In [ ]:
hfss.disconnect_ansys()